In [2]:
import pandas as pd
import re

In [9]:
# Paths to your files
input_csv = "test.csv"    # change to your file path
output_csv = "test_clean1.csv"

# Load CSV
df = pd.read_csv(input_csv)

# Function to extract Value and Unit
def extract_value_unit(text):
    # Extract Value
    value_match = re.search(r"Value:\s*([\d\.]+)", text)
    value = float(value_match.group(1)) if value_match else None

    # Extract Unit
    unit_match = re.search(r"Unit:\s*([\w\s]+)", text)
    unit = unit_match.group(1).strip() if unit_match else None

    # Remove Value and Unit lines from catalog_content
    text_clean = re.sub(r"Value:\s*[\d\.]+\s*", "", text)
    text_clean = re.sub(r"Unit:\s*[\w\s]+\s*", "", text_clean)

    return pd.Series([value, unit, text_clean])

# Apply extraction
df[['value', 'unit', 'catalog_content']] = df['catalog_content'].apply(extract_value_unit)

# Convert unit to lowercase
df['unit'] = df['unit'].str.lower()

# Mapping of unit variants → standard unit and conversion factor
unit_mapping = {
    # Weight variants
    'oz': ('ounce', 1),
    'ounce': ('ounce', 1),
    'ounces': ('ounce', 1),
    'lb': ('ounce', 16),
    'pound': ('ounce', 16),
    'pounds': ('ounce', 16),
    'g': ('ounce', 0.03527396),
    'gram': ('ounce', 0.03527396),
    'grams': ('ounce', 0.03527396),
    'kg': ('ounce', 35.27396),
    'gramm': ('ounce', 0.03527396),

    # Volume variants
    'fl oz': ('fl oz', 1),
    'fl ounce': ('fl oz', 1),
    'floz': ('fl oz', 1),
    'fluid ounce': ('fl oz', 1),
    'fluid ounces': ('fl oz', 1),
    'ml': ('fl oz', 0.033814),
    'millilitre': ('fl oz', 0.033814),
    'milliliter': ('fl oz', 0.033814),
    'mililitro': ('fl oz', 0.033814),
    'millilitro': ('fl oz', 0.033814),
    'milliliters': ('fl oz', 0.033814),
    'liters': ('fl oz', 33.814),
    'liter': ('fl oz', 33.814),
    'l': ('fl oz', 33.814),
    'ltr': ('fl oz', 33.814),
    'fl': ('fl oz', 1),  # approximate, depends on context

    # Count / each
    'ct': ('count', 1),
    'each': ('count', 1),
    'piece': ('count', 1),
    'pieces': ('count', 1),

    'pack': ('packs', 1)
}

# Function to standardize unit and adjust value
def standardize_unit(row):
    unit = str(row['unit']).lower().strip()
    value = row['value']
    
    if unit in unit_mapping:
        standard_unit, factor = unit_mapping[unit]
        new_value = value * factor
        return pd.Series([standard_unit, new_value])
    else:
        # Keep unknown units as-is
        return pd.Series([unit, value])

# Apply standardization and directly replace columns
df[['unit', 'value']] = df.apply(standardize_unit, axis=1)

# Save cleaned and sorted CSV
df.to_csv(output_csv, index=False)

print(f"Cleaned, sorted Units standardized and replaced CSV saved to: {output_csv}")


Cleaned, sorted Units standardized and replaced CSV saved to: test_clean1.csv


In [10]:
# Load cleaned CSV
csv_file = "test_clean1.csv"  # your cleaned file
df = pd.read_csv(csv_file)

# Count occurrences of each unit
unit_counts = df['unit'].dropna().value_counts()

print("Unit counts and parity:")
for unit, count in unit_counts.items():
    print(f"{unit}: {count}")

Unit counts and parity:
ounce: 44173
count: 18253
fl oz: 11436
none: 1011
packs: 36
bottle: 9
bag: 7
1: 6
box: 5
product_weight: 5
foot: 4
tea bags: 4
jar: 4
packet: 3
k: 3
paper cupcake liners: 3
ea: 3
sq ft: 2
lbs: 2
per package: 2
pac: 2
gallon: 2
na: 1
kit: 1
200 capsules: 1
fluid_ounces: 1
16 ounces: 1
2: 1
container: 1
sachet: 1
unit: 1
0: 1
m: 1
tin: 1
case: 1
jars: 1
gallons: 1
cou: 1
sugar substitute: 1
pouch: 1
capsules: 1
stück: 1
comes as a single dispenser unit with 1 count of candy refills
bullet point 5: 1
12: 1
16: 1


In [11]:
# Load the dataset
df = pd.read_csv("test_clean1.csv")

# Function to extract IPQ from catalog_content
def extract_ipq(text):
    if pd.isna(text):
        return 1  # default to 1 if not found

    # Common patterns for pack quantity
    patterns = [
        r'pack of (\d+)',          # "Pack of 10"
        r'(\d+)\s*packs',         # "10 packs"
        r'packs?\s*(\d+)',        # "packs 10" or "pack 10"
        r'(\d+)\s*per pack',      # "12 per pack"
        r'(\d+)\s*per case',      # "12 per case"
        r'(\d+)\s*pack',          # "10 pack"
        r'(\d+)\s*count',         # "20 count"
        r'(\d+)\s*pieces',        # "240 pieces"
        r'(\d+)\s*packet',        # "24 packets"
        r'(\d+)[-/]carton',       # "6/carton" or "6-carton"
        r'case\s*(\d+)',          # "case 6"
    ]
    
    for pat in patterns:
        match = re.search(pat, text, re.IGNORECASE)
        if match:
            return int(match.group(1))
    return 1  # default to 1 if nothing matches

# Function to extract item_name and specification
def extract_fields(text):
    if pd.isna(text):
        return pd.Series({'item_name': None, 'specification': None})
    
    # Item Name
    item_match = re.search(r'Item Name:\s*(.*?)(?:\n|$)', text)
    item_name = item_match.group(1).strip() + '\n' if item_match else None
    
    # Specification (combine bullet points and product description)
    bullets = re.findall(r'Bullet Point \d+:\s*(.*?)(?:\n|$)', text)
    bullet_text = " ".join([b.strip() for b in bullets]) if bullets else ""
    
    desc_match = re.search(r'Product Description:\s*(.*?)(?:\n|$)', text)
    desc_text = desc_match.group(1).strip() if desc_match else ""
    
    # Combine bullet points and description into specification
    specification = (bullet_text + " " + desc_text).strip() + '\n' if (bullet_text or desc_text) else None
    
    return pd.Series({'item_name': item_name, 'specification': specification})

# Apply extraction
df['ipq'] = df['catalog_content'].apply(extract_ipq)
parsed_cols = df['catalog_content'].apply(extract_fields)
df = pd.concat([df, parsed_cols], axis=1)

# Define the desired column order
column_order = [
    'sample_id',
    'item_name',
    'specification',
    'ipq',
    'unit',
    'value',
    'image_link',
    'catalog_content'
]

# Reorder the DataFrame
df = df[column_order]

# Save to CSV
df.to_csv("test_clean2.csv", index=False)

# Confirmation
print("test_clean2.csv saved with the new column order!")

test_clean2.csv saved with the new column order!


In [12]:
test_df = pd.read_csv("test_clean2.csv")
print(test_df.columns)
print(test_df.isnull().sum())  # Check for missing values

Index(['sample_id', 'item_name', 'specification', 'ipq', 'unit', 'value',
       'image_link', 'catalog_content'],
      dtype='object')
sample_id              0
item_name              8
specification      13707
ipq                    0
unit                   2
value               1011
image_link             0
catalog_content        0
dtype: int64


In [15]:
def clean_text(text):
    if pd.isna(text):
        return text
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower().strip()  # Lowercase and remove extra spaces
    text = text.replace('\n', ' ')  # Replace newlines with spaces
    return text

for col in ['item_name', 'specification', 'catalog_content']:
    test_df[col] = test_df[col].apply(clean_text)

# Text columns
test_df['item_name'].fillna('', inplace=True)
test_df['specification'].fillna('', inplace=True)
test_df['catalog_content'].fillna('', inplace=True)

test_df['unit'].fillna('unknown', inplace=True)

test_df['value'].fillna(test_df['value'].median(), inplace=True)

test_df.to_csv("test_clean3.csv", index=False)

C:\Users\aryan\AppData\Local\Temp\ipykernel_17672\784390163.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['item_name'].fillna('', inplace=True)
C:\Users\aryan\AppData\Local\Temp\ipykernel_17672\784390163.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [3]:
# Load dataset
df = pd.read_csv("test_clean1.csv")

# Function to extract item_name and specification
def extract_fields(text):
    if pd.isna(text):
        return pd.Series({'item_name': None, 'specification': None})
    
    # Extract Item Name
    item_match = re.search(r'Item Name:\s*(.*?)(?:\n|$)', text, re.IGNORECASE)
    item_name = item_match.group(1).strip() if item_match else None
    
    # Remove the entire Item Name line from text
    text_without_item_name = re.sub(r'Item Name:.*(?:\n|$)', '', text, flags=re.IGNORECASE)
    
    # Remove remaining tags (Bullet Points, Product Description)
    specification_text = re.sub(
        r'Bullet Point \d+:|Product Description:', 
        '', 
        text_without_item_name, 
        flags=re.IGNORECASE
    ).strip()
    
    return pd.Series({'item_name': item_name, 'specification': specification_text})

# Apply extraction
parsed_cols = df['catalog_content'].apply(extract_fields)
df = pd.concat([df, parsed_cols], axis=1)

# Drop the original catalog_content column
df = df.drop(columns=['catalog_content'])

# Save to CSV
df.to_csv("test_clean4.csv", index=False)
print("test_clean4.csv saved with only item_name and specification!")


test_clean4.csv saved with only item_name and specification!


In [4]:
# Load the CSV
df = pd.read_csv("test_clean4.csv")

# Function to clean text
def clean_text(text):
    if pd.isna(text):
        return ""
    # Convert to lowercase
    text = text.lower()
    # Keep only letters, numbers, and spaces; replace other chars with space
    text = re.sub(r'[^a-z0-9 ]+', ' ', text)
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning to desired columns
for col in ['item_name', 'specification']:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

# Save cleaned CSV
df.to_csv("test_clean4.csv", index=False)
print("test_clean4.csv saved with lowercase and cleaned text!")

test_clean4.csv saved with lowercase and cleaned text!


In [7]:
# Load the cleaned CSV
df = pd.read_csv("test_clean4.csv")

# Define quantity regex patterns
quantity_patterns = [
    r'pack\s*of\s*(\d+)',
    r'(\d+)\s*packs?',
    r'packs?\s*(\d+)',
    r'(\d+)\s*per\s*pack',
    r'(\d+)\s*per\s*case',
    r'(\d+)\s*count',
    r'(\d+)\s*ct',
    r'(\d+)\s*pieces?',
    r'(\d+)\s*pcs',
    r'(\d+)\s*piece',
    r'(\d+)\s*pc',
    r'(\d+)\s*packet[s]?',
    r'(\d+)[-/]\s*carton[s]?',
    r'case\s*(\d+)',
    r'pk[-\s]*(\d+)',
    r'(\d+)[-\s]*pk',
    r'(\d+)\s*x\s*(\d+)?',
    r'(\d+)\s*bag[s]?',
    r'(\d+)\s*box(?:es)?',
    r'(\d+)\s*bottle[s]?',
    r'(\d+)\s*can[s]?',
    r'(\d+)\s*tube[s]?',
    r'(\d+)\s*sachet[s]?',
    r'(\d+)\s*roll[s]?',
    r'(\d+)\s*pair[s]?',
    r'(\d+)\s*set[s]?',
    r'size[:\s-]*(\d+)',
]

# Function to extract quantity from item_name
def extract_quantity(text):
    if pd.isna(text):
        return None
    for pattern in quantity_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            # If pattern has two groups (like "2 x 3"), multiply them
            if len(match.groups()) > 1 and match.group(2):
                return int(match.group(1)) * int(match.group(2))
            return int(match.group(1))
    return None  # If nothing matches

# Apply extraction
df['quantity'] = df['item_name'].apply(extract_quantity)

# Save to new CSV
df.to_csv("test_clean5.csv", index=False)
print("test_clean5.csv saved with new 'quantity' column!")

test_clean5.csv saved with new 'quantity' column!


In [8]:
column_order = [
    'sample_id',
    'item_name',
    'specification',
    'quantity',
    'unit',
    'value',
    'image_link'
]
df = df[[col for col in column_order if col in df.columns]]
df.to_csv("test_clean5.csv", index=False)
print("test_clean5.csv saved with proper column order!")

test_clean5.csv saved with proper column order!


In [2]:
import re
import pandas as pd
import numpy as np

# ============================================================
# 1️⃣ Load the dataset
# ============================================================
df = pd.read_csv("test_clean6.csv")
print(f"Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")

# ============================================================
# 2️⃣ Initialize new columns
# ============================================================
df["weight_oz"] = np.nan
df["volume_fl_oz"] = np.nan
df["count"] = np.nan

# ============================================================
# Weight units → convert everything to ounces (oz)
# ============================================================
weight_units = {
    "oz": 1.0, "ounce": 1.0, "ounces": 1.0,
    "lb": 16.0, "lbs": 16.0, "pound": 16.0, "pounds": 16.0,
    "g": 0.03527396, "gram": 0.03527396, "grams": 0.03527396, "gramm": 0.03527396,
    "kg": 35.27396, "kilogram": 35.27396, "kilograms": 35.27396,
    "mg": 3.5274e-5, "milligram": 3.5274e-5, "milligrams": 3.5274e-5,
    "stone": 224.0, "st": 224.0,           # UK weight
    "ton": 32000.0, "tons": 32000.0        # US short ton
}

# ============================================================
# Volume units → convert everything to fluid ounces (fl oz)
# ============================================================
volume_units = {
    "fl oz": 1.0, "floz": 1.0, "fluid ounce": 1.0, "fluid ounces": 1.0, "fl ounce": 1.0,
    "ml": 0.033814, "milliliter": 0.033814, "millilitre": 0.033814, "mililitro": 0.033814,
    "l": 33.814, "ltr": 33.814, "liter": 33.814, "litre": 33.814, "liters": 33.814, "litres": 33.814,
    "gallon": 128.0, "gallons": 128.0, "gal": 128.0,
    "pt": 16.0, "pint": 16.0, "pints": 16.0,         # US pint
    "qt": 32.0, "quart": 32.0, "quarts": 32.0,     # US quart
    "cup": 8.0, "cups": 8.0,                        # US cup
    "tbsp": 0.5, "tablespoon": 0.5, "tablespoons": 0.5,
    "tsp": 0.1666667, "teaspoon": 0.1666667, "teaspoons": 0.1666667
}

count_keywords = [
    "pack", "packs", "set", "sets", "box", "boxes",
    "carton", "cartons", "bottle", "bottles",
    "can", "cans", "jar", "jars",
    "tube", "tubes", "sheet", "sheets",
    "piece", "pieces", "pcs", "count"
]

# ============================================================
# 4️⃣ Helper: Extract numeric value + unit from text
# ============================================================
def extract_value_unit(text):
    if not isinstance(text, str):
        return None, None
    text = text.lower()
    match = re.search(r'(\d+(?:\.\d+)?)\s*([a-zA-Z ]+)', text)
    if match:
        value, unit = match.groups()
        return float(value), unit.strip()
    return None, None

# ============================================================
# 5️⃣ Pass 1: Use existing `value` and `units` columns (if exist)
# ============================================================
if "value" in df.columns and "units" in df.columns:
    for i, row in df.iterrows():
        val, unit = row["value"], str(row["units"]).lower()

        # Weight units
        for wu in weight_units:
            if wu in unit:
                df.at[i, "weight_oz"] = float(val) * weight_units[wu]
                break

        # Volume units
        for vu in volume_units:
            if vu in unit:
                df.at[i, "volume_fl_oz"] = float(val) * volume_units[vu]
                break

        # Count (like 10 pack, 4 pcs)
        if "pack" in unit or "piece" in unit or "count" in unit or "pcs" in unit:
            df.at[i, "count"] = float(val)

# ============================================================
# 6️⃣ Pass 2: Fill missing from `specification` and `item_name`
# ============================================================
for idx, row in df.iterrows():
    if pd.notna(row["weight_oz"]) and pd.notna(row["volume_fl_oz"]) and pd.notna(row["count"]):
        continue  # skip filled ones

    text = f"{row.get('specification', '')} {row.get('item_name', '')}".lower()

    # --- Check weight ---
    if pd.isna(row["weight_oz"]):
        m = re.search(r'(\d+(?:\.\d+)?)\s*(g|kg|mg|lb|ounce|oz)\b', text)
        if m:
            val, unit = m.groups()
            df.at[idx, "weight_oz"] = float(val) * weight_units[unit]

    # --- Check volume ---
    if pd.isna(row["volume_fl_oz"]):
        m = re.search(r'(\d+(?:\.\d+)?)\s*(ml|l|liter|litre|gallon|fl oz|floz)\b', text)
        if m:
            val, unit = m.groups()
            df.at[idx, "volume_fl_oz"] = float(val) * volume_units[unit]

    # --- Check count ---
    if pd.isna(row["count"]):
        m = re.search(r'(?:pack|set|box|pcs|pieces|count|bottles?)\s*(?:of\s*)?(\d+)', text)
        if m:
            df.at[idx, "count"] = float(m.group(1))
        else:
            # Also handle leading numeric e.g. "12 pack cookies"
            m2 = re.search(r'(\d+)\s*(pack|pcs|pieces|count|bottles?)', text)
            if m2:
                df.at[idx, "count"] = float(m2.group(1))

# Fill remaining missing counts as 1 (future-proof way)
df["count"] = df["count"].fillna(1)

# ============================================================
# 7️⃣ Save updated file
# ============================================================
output_path = "test_clean7.csv"
df.to_csv(output_path, index=False)
print(f"Saved updated dataset → {output_path}")

# Quick stats
print("\n📊 Summary:")
print("Filled weight_oz:", df["weight_oz"].notna().sum())
print("Filled volume_fl_oz:", df["volume_fl_oz"].notna().sum())
print("Filled count:", df["count"].notna().sum())

Loaded data: 75000 rows, 7 columns
Saved updated dataset → test_clean7.csv

📊 Summary:
Filled weight_oz: 50875
Filled volume_fl_oz: 9012
Filled count: 75000


In [3]:
import pandas as pd

# === Detailed category dictionary (duplicates merged) ===
detailed_categories = {
    "tea": ["tea", "teabags", "oolong", "green", "herbal", "chai", "matcha", "brewing", "yerba", "caffeine"],
    "coffee": ["coffee", "espresso", "arabica", "cappuccino", "latte", "brewing", "keurig"],
    "soft_drink": ["soda", "cola", "sparkling", "juice", "energy", "iced", "cold", "smoothies", "cider"],
    "juice": ["juice", "lemonade", "orange", "apple", "fruit", "berry", "tropical", "grape", "citrus"],
    "sweets": ["chocolate", "candy", "chocolates", "bars", "brownie", "lollipops", "oreo", "toffee", "chewy", "lollipop"],
    "cookies": ["cookies", "biscuit", "biscuits", "wafer", "shortbread", "muffin", "cupcake", "cracker"],
    "protein_health": ["protein", "supplement", "chia", "xylitol", "unsweetened", "plant", "pistachio", "superfood", "wellness"],
    "spices": ["spice", "spices", "seasoning", "marinades", "saffron", "cumin", "garlic", "pepper", "herb", "curry", "seasoned"],
    "milk": ["milk", "cream", "butter", "parmesan", "cheese", "yogurt", "dairy"],
    "dry_fruits": ["nuts", "almond", "cashews", "peanuts", "pistachio", "trail", "mix", "seeds", "nutty"],
    "grains": ["grains", "cereal", "oats", "semolina", "barley", "quaker", "breakfast"],
    "noodles": ["pasta", "noodles", "spaghetti", "ramen", "macaroni"],
    "cake": ["cake", "pastry", "donut", "brownie", "cupcake", "muffin"],
    "fruits": ["fruit", "apple", "orange", "lemon", "berry", "banana", "grape", "strawberry", "kiwi", "pineapple", "mango"],
    "vegetables": ["vegetable", "veggies", "tomato", "onion", "corn", "potato", "broccoli", "peppers"],
    "non_veg": ["meat", "beef", "chicken", "pork", "turkey", "eggs", "poultry", "bacon", "sausage"],
    "crispy": ["chips", "crispy", "puffs", "crisps", "snacks", "popcorn"],
    "sauces": ["sauce", "sauces", "ketchup", "mustard", "vinegar", "dressing", "salsa", "paste", "condensed", "broth"],
    "gifts": ["gift", "multi", "canister", "party", "holiday", "christmas", "valentine"],
    "spicy": ["spicy", "chili", "pepper", "masala", "peri peri"],
    "savory": ["salted", "herb", "garlic", "onion", "tangy"]
}

# === Function to extract categories ===
def extract_types(text):
    text = str(text).lower()
    matched_types = []
    for category, keywords in detailed_categories.items():
        for kw in keywords:
            if kw in text:
                matched_types.append(category)
                break
    return " ".join(matched_types) if matched_types else "other"

# === Main processing function ===
def process_csv(input_csv, output_csv):
    df = pd.read_csv(input_csv)

    # 'type' from item_name
    df['type'] = df['item_name'].apply(extract_types)

    # 'related' from specification
    df['related'] = df['specification'].apply(extract_types)

    # Save new CSV
    df.to_csv(output_csv, index=False)
    print(f"✅ Processed CSV saved to {output_csv}")

    # Count occurrences for type
    type_counts = {}
    for types in df['type']:
        for t in types.split():
            type_counts[t] = type_counts.get(t, 0) + 1

    print("\n📊 Number of rows under each TYPE:")
    for t, count in sorted(type_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{t:20s}: {count}")

    # Count occurrences for related
    related_counts = {}
    for rels in df['related']:
        for r in rels.split():
            related_counts[r] = related_counts.get(r, 0) + 1

    print("\n📊 Number of rows under each RELATED:")
    for r, count in sorted(related_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{r:20s}: {count}")

# === Usage ===
input_csv = "test_clean7.csv"
output_csv = "test_clean8.csv"
process_csv(input_csv, output_csv)

✅ Processed CSV saved to test_clean8.csv

📊 Number of rows under each TYPE:
other               : 14492
fruits              : 11255
tea                 : 10491
soft_drink          : 10332
juice               : 10138
sweets              : 9677
spices              : 9061
dry_fruits          : 8698
milk                : 8107
sauces              : 6047
coffee              : 5523
vegetables          : 4993
crispy              : 4619
protein_health      : 4609
savory              : 4223
spicy               : 4019
non_veg             : 3617
gifts               : 3607
cookies             : 3250
grains              : 2574
cake                : 2502
noodles             : 1956

📊 Number of rows under each RELATED:
soft_drink          : 19813
milk                : 19254
other               : 17821
tea                 : 16375
fruits              : 15885
juice               : 15717
dry_fruits          : 15578
spices              : 15325
protein_health      : 12947
vegetables          : 12573
sweets 

In [4]:
nutritional_dict = {
    "protein": ["protein", "whey", "soy", "lentil", "chickpea", "egg", "meat", "milk"],
    "fiber": ["fiber", "multigrain", "wholegrain", "oats", "bran"],
    "sugar": ["sugar", "sweet", "honey", "syrup", "caramel", "candy", "chocolate"],
    "fat": ["butter", "oil", "cheese", "ghee", "cream", "nut", "almond", "cashew"],
    "carbs": ["rice", "pasta", "bread", "noodle", "cereal", "corn", "flour"],
    "vitamins": ["fortified", "enriched", "vitamin", "omega", "probiotic"],
    "low_sugar": ["sugar free", "no sugar", "zero sugar"],
    "low_fat": ["low fat", "fat free", "skimmed"]
}
df = pd.read_csv("test_clean8.csv")
def extract_tags(text, dictionary):
    tags = []
    text = str(text).lower()
    for tag, keywords in dictionary.items():
        for kw in keywords:
            if kw in text:
                tags.append(tag)
                break
    return " ".join(tags) if tags else ""
df["nutritional_value"] = df["item_name"].apply(lambda x: extract_tags(x, nutritional_dict)) + " " + \
                          df["specification"].apply(lambda x: extract_tags(x, nutritional_dict))
df.to_csv("test_clean8.csv", index=False)

print("Added 'nutritional_value' column!")

Added 'nutritional_value' column!


In [5]:
import pandas as pd
import re

# === Load CSV ===
df = pd.read_csv("test_clean8.csv")  # your updated CSV

# Replace commas and multiple spaces with single space
df['type'] = df['type'].apply(lambda x: re.sub(r'[,\s]+', ' ', str(x)).strip())
df['related'] = df['related'].apply(lambda x: re.sub(r'[,\s]+', ' ', str(x)).strip())
df['nutritional_value'] = df['nutritional_value'].apply(lambda x: re.sub(r'[,\s]+', ' ', str(x)).strip())

# Save the cleaned CSV
df.to_csv("test_clean8.csv", index=False)
print("Saved CSV with cleaned type column")

Saved CSV with cleaned type column
